### Runtime setup
The following envs enable stable retries and quiet streaming.

- `SCILLM_FORCE_HTTPX_STREAM=1`
- `LITELLM_MAX_RETRIES=3`, `LITELLM_RETRY_AFTER=1`, `LITELLM_TIMEOUT=45`
- Requires `tenacity` installed for backoff.

In [1]:
import os
os.environ.setdefault('SCILLM_FORCE_HTTPX_STREAM','1')
os.environ.setdefault('LITELLM_MAX_RETRIES','3')
os.environ.setdefault('LITELLM_RETRY_AFTER','1')
os.environ.setdefault('LITELLM_TIMEOUT','45')
try:
    import tenacity  # noqa: F401
    print('tenacity: ok')
except Exception:
    print('tenacity missing — run: pip install tenacity')


tenacity: ok


# Provider Matrix — OpenAI, Anthropic, Perplexity, Chutes

        Runs a minimal inference for common providers and a Chutes call with attribution.
        Only executes sections where required env vars are present.

In [2]:
import os, json
from scillm import completion
from scillm.extras import infer_with_fallback

def _print_resp(tag, resp):
    try:
        content = resp.choices[0].message.get('content','')
    except Exception:
        content = str(resp)
    served = getattr(resp,'model', None) or (resp.get('model') if isinstance(resp,dict) else None)
    print(f"[{tag}] content=", content)
    if served:
        print(f"[{tag}] served_model=", served)

# OpenAI
ok = os.environ.get('OPENAI_API_KEY')
if ok:
    try:
        r = completion(
            model=os.environ.get('OPENAI_MODEL','gpt-4o-mini'),
            custom_llm_provider='openai',
            api_key=ok,
            messages=[{'role':'user','content':'In one word, say OK'}],
            max_tokens=8, temperature=0,
        )
        _print_resp('openai', r)
    except Exception as e:
        print('[openai] error —', str(e)[:160])
else:
    print('[openai] OPENAI_API_KEY not set — skipped')

# Anthropic
ak = os.environ.get('ANTHROPIC_API_KEY')
if ak:
    try:
        r = completion(
            model=os.environ.get('ANTHROPIC_MODEL','claude-3-haiku-20240307'),
            custom_llm_provider='anthropic',
            api_key=ak,
            messages=[{'role':'user','content':'In one word, say OK'}],
            max_tokens=8, temperature=0,
        )
        _print_resp('anthropic', r)
    except Exception as e:
        print('[anthropic] error —', str(e)[:160])
else:
    print('[anthropic] ANTHROPIC_API_KEY not set — skipped')

# Perplexity
pk = os.environ.get('PERPLEXITY_API_KEY')
if pk:
    try:
        r = completion(
            model=os.environ.get('PERPLEXITY_MODEL','sonar'),
            custom_llm_provider='perplexity',
            api_key=pk,
            messages=[{'role':'user','content':'In one word, say OK'}],
            max_tokens=8, temperature=0,
        )
        _print_resp('perplexity', r)
    except Exception as e:
        print('[perplexity] error —', str(e)[:160])
else:
    print('[perplexity] PERPLEXITY_API_KEY not set — skipped')

# Chutes (OpenAI-compatible) with attribution-first fallback
cb = os.environ.get('CHUTES_API_BASE_1') or os.environ.get('CHUTES_API_BASE')
ck = os.environ.get('CHUTES_API_KEY_1') or os.environ.get('CHUTES_API_KEY')
if cb and ck:
    r, meta = infer_with_fallback(
        messages=[{'role':'user','content':'Return only {"ok":true} as JSON.'}],
        kind='text', require_json=True,
        max_retries=3, retry_after=1, timeout=45,
    )
    _print_resp('chutes', r)
    print('[chutes] meta=', json.dumps(meta, indent=2))
else:
    print('[chutes] CHUTES_API_BASE/CHUTES_API_KEY not set — skipped')

[openai] OPENAI_API_KEY not set — skipped



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

[anthropic] error — litellm.AuthenticationError: AnthropicException - {"type":"error","error":{"type":"authentication_error","message":"invalid x-api-key"},"request_id":"req_011CUY


[perplexity] content= OK.
[perplexity] served_model= perplexity/sonar


[chutes] content= {"ok": true}
[chutes] served_model= openai_like/Alibaba-NLP/Tongyi-DeepResearch-30B-A3B
[chutes] meta= {
  "routing": "router",
  "preferred": "chutes-1:Alibaba-NLP/Tongyi-DeepResearch-30B-A3B",
  "served_model": "openai_like/Alibaba-NLP/Tongyi-DeepResearch-30B-A3B",
  "attempts": []
}
